<a href="https://colab.research.google.com/github/bigyansapkota97/Covid-vs-Stock-Price/blob/main/Python_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpha_vantage

In [1]:
import pandas as pd

url_conf_cases = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

df_conf_cases_initial = pd.read_csv(url_conf_cases)
df_deaths_initial = pd.read_csv(url_deaths)

#converting date columns to datetime format
date_columns = df_conf_cases_initial.columns[4:]
date_range = pd.to_datetime(date_columns)

# getting data from February 2020 to August 2021
mask = (date_range >= '2020-02-01') & (date_range <= '2021-08-31')
df_conf_cases = df_conf_cases_initial.iloc[:, :4].join(df_conf_cases_initial.iloc[:, 4:][date_columns[mask]])
df_deaths = df_deaths_initial.iloc[:, :4].join(df_deaths_initial.iloc[:, 4:][date_columns[mask]])

# Global level sum of confirmed cases and deaths
global_confirmed = df_conf_cases.iloc[:, 4:].sum(axis=0)
global_deaths = df_deaths.iloc[:, 4:].sum(axis=0)

# Global aggregated dataframe
global_df = pd.DataFrame({
    'Date': global_confirmed.index,
    'Confirmed Cases': global_confirmed.values,
    'Deaths': global_deaths.values
})

print(global_df)

        Date  Confirmed Cases   Deaths
0     2/1/20            12038      260
1     2/2/20            16787      364
2     2/3/20            19887      428
3     2/4/20            23899      494
4     2/5/20            27644      566
..       ...              ...      ...
573  8/27/21        215988154  4513136
574  8/28/21        216545334  4521572
575  8/29/21        216992164  4528394
576  8/30/21        217664015  4537965
577  8/31/21        218284677  4547280

[578 rows x 3 columns]
